# Fake News Classifier Using Bidirectional LSTM

In [50]:
import pandas as pd

In [51]:
df=pd.read_csv('C:\\Users\\Skouza\\Desktop\\ML projects\\4-Fake News Classifier\\data\\train.csv')

In [52]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [53]:
#Drop Nan Values
df=df.dropna()


In [54]:
# Get the Independent Features
X=df.drop('label',axis=1)

In [55]:
# Get the Dependent features
y=df['label']

In [56]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [57]:
X.shape

(18285, 4)

In [58]:
y.shape

(18285,)

In [59]:
import tensorflow as tf

In [60]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [61]:
# Vocabulary size
voc_size=5000

### Onehot Representation

In [62]:
messages=X.copy()

In [63]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [64]:
#we reset index bacause we've done a dropna 
messages.reset_index(inplace=True)

In [65]:
import nltk
import re
from nltk.corpus import stopwords

In [66]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Skouza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [68]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [69]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[608, 3386, 2569, 781, 63, 2467, 2447, 918, 4368, 3950],
 [2480, 311, 3817, 2145, 2913, 2681, 3170],
 [3600, 3988, 4097, 733],
 [2265, 932, 1052, 4684, 3642, 2875],
 [3700, 2913, 306, 375, 1458, 1849, 2913, 1316, 284, 3537],
 [1908,
  4617,
  2139,
  3608,
  1567,
  225,
  692,
  2661,
  1467,
  4948,
  4414,
  4833,
  2082,
  1535,
  3170],
 [4407, 1400, 3086, 4903, 4595, 274, 4563, 1862, 3450, 367, 1882],
 [3098, 4617, 312, 2922, 1134, 3953, 225, 4003, 3450, 367, 1882],
 [4024, 2715, 1877, 700, 1008, 2951, 1002, 4335, 225, 61],
 [2501, 1745, 2578, 2433, 1988, 3857, 2706, 3794],
 [1955, 649, 639, 3017, 4960, 4802, 2611, 3290, 3184, 1927, 3413],
 [4684, 3921, 63, 2951, 225, 1134],
 [2793, 408, 4720, 2981, 1904, 4632, 2413, 2633, 3633],
 [1093, 3619, 528, 4117, 2436, 3918, 3551, 3450, 367, 1882],
 [2752, 4953, 1268, 4458, 3128, 3450, 367, 1882],
 [2972, 4494, 759, 4087, 4735, 1335, 2707, 3512, 1641, 989],
 [3748, 96, 311],
 [3590, 417, 886, 2001, 225, 1327, 4617, 3170],
 [1542, 723, 38

### Embedding Representation

In [70]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  918 4368 3950]
 [   0    0    0 ... 2913 2681 3170]
 [   0    0    0 ... 3988 4097  733]
 ...
 [   0    0    0 ... 3450  367 1882]
 [   0    0    0 ...  865 1323 4539]
 [   0    0    0 ... 4056 2385 1594]]


In [94]:
## Creating LSTM model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_4 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [73]:
len(embedded_docs),y.shape

(18285, (18285,))

In [74]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [75]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [82]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 12ms/step - loss: 0.3348 - accuracy: 0.8440 - val_loss: 0.2044 - val_accuracy: 0.9122
Epoch 2/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1405 - accuracy: 0.9441 - val_loss: 0.1941 - val_accuracy: 0.9213
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0964 - accuracy: 0.9633 - val_loss: 0.2093 - val_accuracy: 0.9200
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0647 - accuracy: 0.9787 - val_loss: 0.2296 - val_accuracy: 0.9171
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0383 - accuracy: 0.9880 - val_loss: 0.2962 - val_accuracy: 0.9109
Epoch 6/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0225 - accuracy: 0.9937 - val_loss: 0.3792 - val_accuracy: 0.9157
Epoch 7/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0124 - accuracy: 0.9962 - val_loss: 0.4668 - val_accuracy: 0.9047

### Performance Metrics And Accuracy

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)

189/189 [==============================] - 1s 3ms/step


In [47]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]], dtype=int64)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



c:\Users\Skouza\Desktop\ML projects\env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Skouza\Desktop\ML projects\env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Skouza\Desktop\ML projects\env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu